In [1]:
import pymongo

def checkrows():
    client = pymongo.MongoClient('mongodb://localhost:27017/')
    db = client['datasets']
    collection = db['camel-ai']
    rows = collection.find()
    for row in rows:
        st = ""
        print("-=--*--=-"*12)
        for key in row.keys():
            print("{key.capitalize()}:")
            st = ""
            for value in row.values():
                if len(st) > 48:
                    print(st)
                    st = ""
                if len(st) != 0:
                    st += ", "
                st += f"{value}"

checkrows()

In [2]:
def mkrow(user, bot, topic, sub):
    return f"""### System:
I am Bot. I am having a conversation with User.
The topic is {topic}, and we are discussing {sub} in particular.
I will pay very close attention to my calculations, logic, and formulae.

### History:
User: {user}

### Response:
Bot: {bot}

<|endoftext|>
"""

In [3]:
import datasets

ds = datasets.load_dataset("camel-ai/math", streaming=True)

x = 0

# Stow to the PyMongo database used in the next cell.
# This will be used to train the model in the next cell.
def mkrow(user, bot, topic, sub):
    return {"user": user, "bot": bot, "topic": topic, "sub": sub}

import pymongo
import os

client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["datasets"]
coll = db["math"]

for example in ds["train"]:
    try:
        #coll.insert_one(mkrow(example["message_1"], example["message_2"], example["topic;"], example["sub_topic"]))
        doc = {
            "user": example["message_1"],
            "bot": example["message_2"],
            "topic": example["topic;"],
            "sub": example["sub_topic"],
            "source": "hf://camel-ai/math"
        }
        coll.insert_one(doc)
        print(f"Processed {x} rows.")
    except Exception as e:
        print(f"Error processing sample {x}\n\n{e}\n\n")
        pass
    x += 1


Error processing sample 0

'topic'


Error processing sample 1

'topic'


Error processing sample 2

'topic'


Error processing sample 3

'topic'


Error processing sample 4

'topic'


Error processing sample 5

'topic'


Error processing sample 6

'topic'




## Using PyMongo

From now on, my raw data will be stored as gzipped base64 strings in MongoDB via PyMongo.

Base64 expands strings by approximately 33% on average.

Gzip reduces the size by 70-90% on average.

This should yield a 37-57% reduction in dataset size, whilst ensuring perfect data storage through lossless compression and encoded binary strings.

The scores and subject are NOT compressed, to allow for quick and easy indexing according the the most obvious metrics.

In [ ]:
import json
import datasets
import gzip
import base64

# @Author Mistral-Medium @ Poe
def compress_string(input_string: str) -> str:
    compressed_data = gzip.compress(input_string.encode(), compresslevel=9)
    base64_encoded_data = base64.b64encode(compressed_data).decode()
    return base64_encoded_data

# @Author Samuel L Meyers
def parseds(url, path, out):
    ds = datasets.load_dataset(url, path, streaming=True)
    x = 0
    missed = 0
    with open(f"{out}.txt", "w", encoding="utf-8") as f:
        for example in ds["train"]:
            try:
                obj = {
                    "user": example["message_1"],
                    "bot": example["message_2"],
                    "topic": example["topic;"],
                    "sub": example["sub_topic"]
                }
                row = mkrow(obj["user"], obj["bot"], obj["topic"], obj["sub"]) + "\n\n\n"
                f.write(row)
                f.flush()
                print(f"Processed {x} rows.")
                log = open("math.log", "a")
                log.write(f"{'-==*==-'*7}Processed {x} rows.\@jsonrow,{json.dumps(obj)}\n{row}\n\n\n")
            except Exception as e:
                print(f"Error processing sample {x} : Missed {missed}.\n\n")
                err = open("math.err.log", "a")
                err.write(f"{'-'*48}Error processing sample {x} : Missed {missed}.\n\n{e}\n\n{'-'*48}\n\n")
                missed += 1
            
            x += 1

import json
import gzip
import base64
from pymongo import MongoClient

def compress_strings(strings: dict[str, str]) -> str:
    compressed_strings = gzip.compress(json.dumps(strings).encode(), compresslevel=9)
    return base64.b64encode(compressed_strings).decode()

def store_arxiv(bot: str, title: str, abstract: str, scores: list[int]):
    # Connect to the MongoDB server
    client = MongoClient('mongodb://localhost:27017/')
    db = client['datasets']
    collection = db['math']

    # Compress the title and abstract strings using gzip and base64 encoding
    strings_to_compress = {'abstract': abstract, 'bot': bot}
    compressed_data = compress_strings(strings_to_compress)

    # Create a document to store in the MongoDB collection
    document = {
        'title': title,
        'scores': scores,
        'data': compressed_data,
        'source': 'hf://math-ai/AutoMathText'
    }

    # Insert the document into the MongoDB collection
    collection.insert_one(document)

def parsearxiv(name):
    ds = datasets.load_dataset("math-ai/AutoMathText", "arxiv-0.70-1.00", streaming=True)
    x = 0
    ms = 0
    try:
        for row in ds["train"]:
            bot = row["text"]
            user = row["abstract"]
            title = row["title"]
            scores = [
                float(row["meta"]["lm_q1_score"]),
                float(row["meta"]["lm_q1q2_score"]),
                float(row["meta"]["lm_q2_score"]),
            ]
            store_arxiv(bot, title, user, scores)
            jsn = json.dumps({"user": user, "bot": bot, "title": title, "scores": scores})
            print("* Stored {x} of ~42.5k rows.\n{'-=--^--=-'*12}\n{jsn}\n\n\n")

    except Exception as e:
        err = open(f"{name}.err.log", "a")
        err.write(f"{'-'*48}Error processing sample {x} : Missed {ms}.\n\n{e}\n\n{'-'*48}\n\n")
        ms += 1
        print(f"Error processing sample {x} : Missed {ms}.\n\n")

AttributeError: partially initialized module 'pandas' has no attribute '_pandas_parser_CAPI' (most likely due to a circular import)